In [13]:
import csv
import re
import string
from konlpy.tag import Komoran

class Preprocess:
    def __init__(self):
        self.komoran = Komoran()
        with open('datasets/stopwords.txt', 'r', encoding='utf-8') as f:
            self.stopwords = set(f.read().split(','))

    def preprocess(self, text):
        text = text.strip()  
        text = re.compile('<.*?>').sub('', text) 
        text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
        text = re.sub('\s+', ' ', text)  
        text = re.sub(r'[^\w\s]', ' ', str(text).strip())
        text = re.sub(r'\d', ' ', text) 
        text = re.sub(r'\s+', ' ', text) 
        return text
    
    def is_korean(self, text):
        # Check if the text contains Hangul characters only
        korean_pattern = re.compile('[^ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+')
        if korean_pattern.search(text):
            return False
        return True
    
    def final(self, text):
        n = []
        word = self.komoran.nouns(text)
        p = self.komoran.pos(text)
        for pos in p:
            if pos[1] in ['SL'] and self.is_korean(pos[0]):
                word.append(pos[0])
        for w in word:
            if len(w) > 1 and w not in self.stopwords:
                n.append(w)
        return " ".join(n)

    def finalpreprocess(self, documents):
        for doc in documents:
            doc['article'] = self.final(self.preprocess(doc['article']))
        return documents

documents = []
with open('datasets/Korea_DB_0413.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        title = row['title']
        article = row['article']
        documents.append({'title': title, 'article': article})

preprocessor = Preprocess()
processed_documents = preprocessor.finalpreprocess(documents)

In [14]:
exclude_words = ['뉴시스', '기자', '뉴스', '동아']
filtered_documents = []

for doc in processed_documents:
    article_words = doc['article'].split()  # 전처리된 텍스트를 공백 기준으로 단어로 분리
    filtered_words = [word for word in article_words if word not in exclude_words]
    filtered_article = ' '.join(filtered_words)  
    if filtered_article.endswith('서울'):
        filtered_article = filtered_article.rsplit(' ', 1)[0]
    doc['article'] = filtered_article
    filtered_documents.append(doc)

filtered_documents

[{'title': '조태열-블링컨 첫 통화…“한미 동맹 더욱 심화·발전”',
  'article': '취임 인사 유선 협의 도발 밀착 규탄 조태열 외교부 장관 오후 안토니 미국 국무장관 통화 협력 북한 문제 협의 취임 인사 유선 협의 외교부 장관 이날 카운터 파트 장관 전화 통화 장관 장관 취임 축하 동맹 발전 소통 장관 사의 지난해 국빈 방미 동맹 주년 성과 발전 나가야 책임감 파트너십 동맹 심화 희망 장관 올해 정상 회의 출범 주년 해임 상기 캠프 데이비드 합의 토대 협력 강화 장관 적극 공감 상호 편리 시기 장관 미국 방문 동맹 협력 강화 방안 심도 협의 장관 이날 통화 새해 첫날 서해 포병 사격 포함 북한 위협 평가 우려 공유 북한 불법 미사일 개발 도발 무기 지원 포함 군사 협력 규탄 유엔 국제무대 공조 인식 협의 그룹 확장 억제 전략 협의체 확장 억제 실효 강화 북한 도발 대응 국제 사회 안전 보장 이사회 안보리 결의 의무 이행 견인'},
 {'title': '조국 “민주, 총선서 포용력 발휘해 반윤연대 꾸려야”',
  'article': '정책 싱크탱크 리셋 코리아 행동 준비 세미나 개최 조국 이낙연 탈당 반대 역할 조국 법무부 장관 총선 윤석열 연대 필요 강조 더불어민주당 적극 역할 주문 장관 서울 종로구 노무현 시민 센터 정책 싱크탱크 리셋 코리아 행동 준비 세미나 리셋 코리아 행동 윤석열 정권 반대 정치 세력 연대 생각 민주당 세력 만큼 적극 포용력 발휘 연대 희망 윤석열 정권 검찰 통치 권력 방탄 계속 방송 장악 시도 비판 언론 강제 수사 국민 투쟁 쟁취 정치 민주주의 강조 근본 위기 진행 저출산 국가 소멸 위기 국가 성장 동력 상실 생존 기반 붕괴 사회 양극 공동체 파괴 지구 기후 위기 강대국 갈등 평화 안보 위기 설명 윤석열 정권 위기 해결 의지 능력 판단 무능 정권 비판 윤석열 정권 반대 정치 목소리 대한민국 위기 극복 정권 교체 방책 국민 제시 리셋 코리아 행동 위기 극복 비전 정책 제시 부연 리셋 코리아 행동 정책 정치 분리 직시

In [15]:
with open('datasets/preprocessed.csv', mode='w', encoding='utf-8', newline='') as file:
    fieldnames = ['title', 'article'] 
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    for doc in documents:
        writer.writerow(doc)